In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
from tqdm.notebook import tqdm
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv', index_col = 0)
test = pd.read_csv('test.csv', index_col = 0)
sub = pd.read_csv('sample_submission.csv', index_col = 0)

In [3]:
train

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
index,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0


In [4]:
px = train.drop('label', axis=1)

In [5]:
label = train['label']

In [6]:
train_x, test_x, train_y, test_y = train_test_split(px, label, test_size=0.25, shuffle=True, stratify=label)

In [7]:
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()
train_y = train_y.to_numpy()
test_y = test_y.to_numpy()

In [8]:
test = test.to_numpy()

In [9]:
def preprocessing(img):
    # 텐서형으로 transform
    transform_train = transforms.Compose([
        #transforms.ToPILImage(),
        #transforms.RandomHorizontalFlip(p=0.5),
        #transforms.RandomVerticalFlip(p=0.5),
        #A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225),max_pixel_value=255.0),
        ## Check transpose_mask=True
        transforms.ToTensor()
        #transforms.Normalize(0.5, 0.5),
        ])
    
    img = transform_train(img)
    img = img.transpose(0,1)
    return img.float()

In [10]:
def preprocessing_test(img):
    # 텐서형으로 
    transform_train = transforms.Compose([
        #transforms.ToPILImage(),
        transforms.ToTensor()
        #transforms.Normalize(0.5, 0.5),
    ])
    
    img = transform_train(img)
    img = img.transpose(0,1)
    return img.float()

In [11]:
class trainDataset(data.Dataset):
    def __init__(self,data,label,):
        super().__init__()
        self.data = data.reshape(-1, 1, 28, 28).astype('float32')
        self.label = label
    def __getitem__(self, index):
        img = self.data[index]
        img = preprocessing(img)
        label = self.label[index]
        return img, label
    def __len__(self):
        return len(self.data)
    

In [12]:
class testDataset(data.Dataset):
    def __init__(self,data,):
        super().__init__()
        self.data = data.reshape(-1, 1, 28, 28).astype('float32')
    def __getitem__(self, index):
        img = self.data[index]
        img = preprocessing_test(img)
        return img
    def __len__(self):
        return len(self.data)

In [13]:
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets as datasets
from torchvision import models

In [14]:
model = models.wide_resnet50_2(pretrained = True)

C:\Users\lmj\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\lmj\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [15]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2),padding=(3, 3), bias=False)
fc_in_features = model.fc.in_features
model.fc = nn.Linear(fc_in_features, out_features=10, bias=True)

In [16]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [18]:
def fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history):

    base_epochs = len(history)
  
    for epoch in range(base_epochs, num_epochs+base_epochs):
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        # 훈련 페이즈
        net.train()
        count = 0

        for inputs, labels in tqdm(train_loader):
            count += len(labels)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 경사 초기화
            optimizer.zero_grad()

            # 예측 함수
            outputs = net(inputs)

            # 손실 함수
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            # 경사 계산
            loss.backward()

            # 파라미터 수정
            optimizer.step()

            # 예측 라벨 산출
            predicted = torch.max(outputs, 1)[1]

            # 정답 건수 산출
            train_acc += (predicted == labels).sum().item()

            # 손실과 정확도 계산
            avg_train_loss = train_loss / count
            avg_train_acc = train_acc / count

        # 예측 페이즈
        net.eval()
        count = 0

        for inputs, labels in test_loader:
            count += len(labels)

            inputs = inputs.to(device)
            labels = labels.to(device)

            # 예측 계산
            outputs = net(inputs)

            # 손실 계산
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # 예측 라벨 산출
            predicted = torch.max(outputs, 1)[1]

            # 정답 건수 산출
            val_acc += (predicted == labels).sum().item()

            # 손실과 정확도 계산
            avg_val_loss = val_loss / count
            avg_val_acc = val_acc / count
    
        print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}], loss: {avg_train_loss:.5f} acc: {avg_train_acc:.5f} val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f}')
        item = np.array([epoch+1, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc])
        history = np.vstack((history, item))
        
    return history

In [20]:
if __name__ == "__main__":
    ds_train = trainDataset(train_x,train_y)
    ds_test = trainDataset(test_x, test_y)
    dl_train = DataLoader(ds_train, batch_size=64, shuffle=True)
    dl_test = DataLoader(ds_test, batch_size=64, shuffle=False)
    print("DATA LOADED")

    model = model.eval().to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    history = np.zeros((0, 5))
    num_epochs = 5
    history = fit(model, optimizer, criterion, num_epochs, dl_train, dl_test, device, history)

DATA LOADED


  0%|          | 0/704 [00:00<?, ?it/s]

Epoch [1/5], loss: 0.00845 acc: 0.80956 val_loss: 0.01911, val_acc: 0.79920


  0%|          | 0/704 [00:00<?, ?it/s]

Epoch [2/5], loss: 0.00783 acc: 0.83238 val_loss: 0.04227, val_acc: 0.68713


...

Epoch [3/5], loss: 0.00720 acc: 0.83887 val_loss: 0.00680, val_acc: 0.84080


  0%|          | 0/704 [00:00<?, ?it/s]

Epoch [4/5], loss: 0.00557 acc: 0.87111 val_loss: 0.00502, val_acc: 0.88373


  0%|          | 0/704 [00:00<?, ?it/s]

Epoch [5/5], loss: 0.00501 acc: 0.88702 val_loss: 0.62013, val_acc: 0.74680


In [21]:
ds_predict = testDataset(test)
dl_predict = DataLoader(ds_predict, batch_size=64, shuffle=False)
pred = []
for inputs in tqdm(dl_predict):
    
    inputs = inputs.to(device) 
    
    outputs = model(inputs)

    # 예측 라벨 산출
    predicted = torch.max(outputs, 1)[1]

    pred.append(predicted)

  0%|          | 0/157 [00:00<?, ?it/s]

In [22]:
pred = torch.cat(pred, dim=0)

In [23]:
pred = pred.cpu()

In [24]:
sub['label'] = pred

In [25]:
sub.to_csv('sub2.csv')

0.88